# Agent-Based Simulation

This notebook contains code examples referring to the Agent-Based Simulation chapter of "Applied Mathematics with Open-Source Software: Operational Research Problems
with Python and R".

First import the required libraries.

In [1]:
import random
import itertools
import numpy as np

Now define a class that defines a City.

In [2]:
class City:
    def __init__(self, size, threshold):
        """Initialises the City object.

        Args:
            size: an integer number of rows and columns
            threshold: float between 0 and 1 representing the
            minimum acceptable proportion of similar neighbours
        """
        self.size = size
        sides = range(size)
        self.coords = itertools.product(sides, sides)
        self.houses = {
            (x, y): House(x, y, threshold, self)
            for x, y in self.coords
        }

    def run(self, n_steps):
        """Runs the simulation of a number of time steps.

        Args:
            n_steps: an integer number of steps
        """
        for turn in range(n_steps):
            self.take_turn()

    def take_turn(self):
        """Swaps all sad households."""
        sad = [h for h in self.houses.values() if h.sad()]
        random.shuffle(sad)
        i = 0
        while i <= len(sad) / 2:
            sad[i].swap(sad[-i])
            i += 1

    def mean_satisfaction(self):
        """Finds the average household satisfaction.

        Returns:
            The average city's household satisfaction
        """
        return np.mean(
            [h.satisfaction() for h in self.houses.values()]
        )

Now define a class that defines a House.

In [3]:
class House:
    def __init__(self, x, y, threshold, city):
        """Initialises the House object.

        Args:
            x: the integer x-coordinate
            y: the integer y-coordinate
            threshold: a number between 0 and 1 representing
              the minimum acceptable proportion of similar
              neighbours
            city: an instance of the City class
        """
        self.x = x
        self.y = y
        self.threshold = threshold
        self.kind = random.choice(["Cardiff", "Swansea"])
        self.city = city

    def satisfaction(self):
        """Determines the household's satisfaction level.

        Returns:
            A proportion
        """
        same = 0
        for x, y in itertools.product([-1, 0, 1], [-1, 0, 1]):
            ax = (self.x + x) % self.city.size
            ay = (self.y + y) % self.city.size
            same += self.city.houses[ax, ay].kind == self.kind
        return (same - 1) / 8

    def sad(self):
        """Determines if the household is sad.

        Returns:
            a Boolean
        """
        return self.satisfaction() < self.threshold

    def swap(self, house):
        """Swaps two households.

        Args:
            house: the house object to swap household with
        """
        self.kind, house.kind = house.kind, self.kind

Now define a function that gives the resulting mean happiness for a given threshold.

In [4]:
def find_mean_happiness(seed, size, threshold, n_steps):
    """Create and run an instance of the simulation.

    Args:
        seed: the random seed to use
        size: an integer number of rows and columns
        threshold: a number between 0 and 1 representing
            the minimum acceptable proportion of similar
            neighbours
        n_steps: an integer number of steps

    Returns:
        The average city's household satisfaction after
        n_steps
    """
    random.seed(seed)
    C = City(size, threshold)
    C.run(n_steps)
    return C.mean_satisfaction()

Find the initial happiness after 0 steps:

In [5]:
initial_happiness = find_mean_happiness(
    seed=0, size=50, threshold=0.65, n_steps=0
)
print(initial_happiness)

0.4998


And the final happiness after 100 steps:

In [6]:
final_happiness = find_mean_happiness(
    seed=0, size=50, threshold=0.65, n_steps=100
)
print(final_happiness)

0.9078
